In [ ]:
# ============ IMPORTS ============
import pandas as pd
import numpy as np
import os
import sys
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add src folder to path
sys.path.insert(0, 'src')
from src.data_download import CACHE_FILE
from src.features import calculate_all_features
from src.scaling import scale_all_features

# ============ CONFIGURATION ============
halving_dates = [
    '2012-11-28',  # First halving: 50 → 25 BTC
    '2016-07-09',  # Second halving: 25 → 12.5 BTC
    '2020-05-11',  # Third halving: 12.5 → 6.25 BTC
    '2024-04-19'   # Fourth halving: 6.25 → 3.125 BTC
]

# Indicators displayed on dashboard
dashboard_indicators = {
    ('Close', 'BTC-USD'): 'blue',
    'RSI': 'red',
    'MACD': 'green',
    'MACD_Signal': 'orange',
    'Fear_and_Greed_Index': 'purple',
    'Cycle_Low_Multiple': 'brown'
}

# Create indicator colors with explicit logic
indicator_colours = {}
for key, colour in dashboard_indicators.items():
    if isinstance(key, str):
        # Already a string, use as-is
        indicator_colours[key] = colour
    else:
        # It's a tuple like ('Close', 'BTC-USD'), extract first element
        indicator_colours[key[0]] = colour

#  ============ AUTO-RELOAD UPDATED MODULES  ============ 
%load_ext autoreload
%autoreload 2

# ============ LOAD CACHED DATA ============
cache = pd.read_pickle(CACHE_FILE)
bitcoin_historical_data = cache['bitcoin_data']
fear_greed_index_data = cache['fgi_data']
start_date = cache['start_date']
end_date = cache['end_date']

print(f"Loaded from {cache['cache_date']}")

# ============ CALCULATE ALL FEATURES ============
# This contains BOTH raw OHLCV AND engineered features (including Cycle_Low_Multiple)
features_df = calculate_all_features(bitcoin_historical_data, fear_greed_index_data)

# ============ PREPARE DATA FOR DASHBOARD ============
# Separate raw OHLCV columns from engineered features
ohlcv_columns = [
    col for col in features_df.columns
    if isinstance(col, tuple) and len(col) == 2 and col[1] == 'BTC-USD'
]

# Extract only engineered features (no raw OHLCV) for scaling
engineered_features = features_df.drop(ohlcv_columns, axis=1, errors='ignore')

# Flatten MultiIndex if present
if isinstance(engineered_features.columns, pd.MultiIndex):
    engineered_features.columns = engineered_features.columns.get_level_values(0)

# Scale the engineered features
scaled_features = scale_all_features(engineered_features)

# For dashboard: we need both raw data (for price/BB) and scaled indicators
bitcoin_data = features_df  # Keep original with OHLCV for price display
absolute_values = scaled_features['absolute_values']  # Scaled features (absolute)
normalised_values = scaled_features['normalised_values']  # Scaled features (0-1)

# ============ DASH APP FOR JUPYTER NOTEBOOK ============

# ============ APP CONFIGURATION ============
app = dash.Dash(__name__)

# Bitcoin halving event dates (convert to datetime)
halving_dates_datetime = [pd.to_datetime(date) for date in halving_dates]

# Indicator dropdown options
indicator_options = [
    {'label': 'None', 'value': 'None'},
    {'label': 'RSI (Absolute)', 'value': 'RSI_abs'},
    {'label': 'RSI (Normalised)', 'value': 'RSI_norm'},
    {'label': 'MACD (Absolute)', 'value': 'MACD_abs'},
    {'label': 'MACD (Normalised)', 'value': 'MACD_norm'},
    {'label': 'MACD Signal (Absolute)', 'value': 'MACD_Signal_abs'},
    {'label': 'MACD Signal (Normalised)', 'value': 'MACD_Signal_norm'},
    {'label': 'Fear & Greed (Absolute)', 'value': 'Fear_and_Greed_Index_abs'},
    {'label': 'Fear & Greed (Normalised)', 'value': 'Fear_and_Greed_Index_norm'},
    {'label': 'Cycle Low Multiple (Absolute)', 'value': 'Cycle_Low_Multiple_abs'},
    {'label': 'Cycle Low Multiple (Normalised)', 'value': 'Cycle_Low_Multiple_norm'},
]

# ============ APP LAYOUT ============
app.layout = html.Div([
    html.H2("Bitcoin Price Cycles with Technical Indicators", 
            style={'textAlign': 'center'}),
    
    # Control panel
    html.Div([
        html.Label("BTC Price Display:"),
        dcc.RadioItems(
            id='btc-scale',
            options=[
                {'label': ' Absolute (USD)', 'value': 'abs'},
                {'label': ' Normalised (0-1)', 'value': 'norm'}
            ],
            value='abs',
            inline=True,
            style={'marginLeft': '10px'}
        ),
        html.Label("Y-Axis Scale:", style={'marginLeft': '30px'}),
        dcc.RadioItems(
            id='y-scale',
            options=[
                {'label': ' Linear', 'value': 'linear'},
                {'label': ' Logarithmic', 'value': 'log'}
            ],
            value='log',
            inline=True,
            style={'marginLeft': '10px'}
        )
    ], style={'textAlign': 'center', 'padding': '10px', 'backgroundColor': '#f0f0f0'}),
    
    # Indicator selectors
    html.Div([
        html.Div([
            html.Label(f"Indicator Row {i}:", style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id=f'indicator-row-{i}',
                options=indicator_options,
                value='None',
                clearable=False,
                style={'margin': 'auto', 'width': '300px'}
            )
        ], style={'textAlign': 'center', 'padding': '5px'})
        for i in [2, 3, 4]
    ]),
    
    dcc.Graph(id='btc-indicator-graph', style={'height': '1400px'})
])

# ============ HELPER FUNCTIONS ============
def add_bitcoin_price_trace(chart_figure, price_scale):
    """Add Bitcoin price trace to top chart row"""
    if price_scale == 'abs':
        # Use raw OHLCV data from features_df
        price_data = bitcoin_data[('Close', 'BTC-USD')]
        trace_name = 'BTC Price'
        hover_template = '<b>BTC Price</b><br>$%{y:,.2f}<extra></extra>'
        line_style = dict(color='blue', width=2)
        
        # Add Bollinger Bands (from raw features_df)
        chart_figure.add_trace(go.Scatter(
            x=bitcoin_data.index,
            y=bitcoin_data['BB_Upper'],
            name='BB Upper',
            line=dict(color='rgba(173, 204, 255, 1.0)', width=2),
            hovertemplate='<b>BB Upper</b><br>$%{y:,.2f}<extra></extra>'
        ), row=1, col=1)
        
        chart_figure.add_trace(go.Scatter(
            x=bitcoin_data.index,
            y=bitcoin_data['BB_Lower'],
            name='BB Lower',
            line=dict(color='rgba(173, 204, 255, 1.0)', width=2),
            fill='tonexty',  # Fill area between upper and lower bands
            fillcolor='rgba(173, 204, 255, 0.3)',
            hovertemplate='<b>BB Lower</b><br>$%{y:,.2f}<extra></extra>'
        ), row=1, col=1)
        
        chart_figure.add_trace(go.Scatter(
            x=bitcoin_data.index,
            y=bitcoin_data['BB_Middle'],
            name='BB Middle (SMA)',
            line=dict(color='rgba(100, 149, 237, 1.0)', width=1, dash='dot'),
            hovertemplate='<b>BB Middle</b><br>$%{y:,.2f}<extra></extra>'
        ), row=1, col=1)
    else:
        # For normalized price, we need to manually normalize it
        price_series = bitcoin_data[('Close', 'BTC-USD')]
        price_data = (price_series - price_series.min()) / (price_series.max() - price_series.min())
        trace_name = 'BTC Price (Norm)'
        hover_template = '<b>BTC Price (Norm)</b><br>%{y:.4f}<extra></extra>'
        line_style = dict(color='blue', width=2, dash='dash')
    
    chart_figure.add_trace(go.Scatter(
        x=bitcoin_data.index, 
        y=price_data, 
        name=trace_name,
        line=line_style, 
        hovertemplate=hover_template
    ), row=1, col=1)

def add_halving_event_lines(chart_figure):
    """Add vertical lines and annotations for Bitcoin halving events"""
    for halving_date in halving_dates_datetime:
        if start_date <= halving_date <= end_date:
            for row_number in [1, 2, 3, 4]:
                chart_figure.add_shape(
                    type="line", 
                    x0=halving_date, 
                    x1=halving_date, 
                    y0=0, 
                    y1=1,
                    xref=f'x{row_number}' if row_number > 1 else 'x',
                    yref=f'y{row_number} domain' if row_number > 1 else 'y domain',
                    line=dict(color="red", width=2, dash="dash"), 
                    opacity=0.6
                )
            # Annotation only on top plot
            chart_figure.add_annotation(
                x=halving_date, 
                y=1.0, 
                xref='x', 
                yref='y domain',
                text="Halving", 
                showarrow=False, 
                yanchor='bottom',
                font=dict(size=10, color='red')
            )

def add_indicator_trace_or_placeholder(chart_figure, row_number, selected_indicator):
    """Add indicator trace or invisible placeholder for empty rows"""
    if selected_indicator == 'None':
        # Invisible dummy trace for empty rows (needed for rangeslider)
        chart_figure.add_trace(go.Scatter(
            x=bitcoin_data.index,
            y=[0] * len(bitcoin_data),
            mode='lines', 
            line=dict(width=0),
            showlegend=False, 
            hoverinfo='skip'
        ), row=row_number, col=1)
    else:
        # Parse indicator name and type (e.g., 'RSI_abs' -> 'RSI', 'abs')
        indicator_name, indicator_type = selected_indicator.rsplit('_', 1)
        
        # Get data and styling based on type
        if indicator_type == 'abs':
            indicator_data = absolute_values[indicator_name]
            line_style = dict(color=indicator_colours[indicator_name], width=2)
        else:
            indicator_data = normalised_values[indicator_name]
            line_style = dict(color=indicator_colours[indicator_name], width=2, dash='dot')
        
        # Add trace to chart
        chart_figure.add_trace(go.Scatter(
            x=indicator_data.index, 
            y=indicator_data,
            name=f'{indicator_name} ({"Abs" if indicator_type == "abs" else "Norm"})',
            line=line_style,
            hovertemplate=f'<b>{indicator_name}</b><br>%{{y:.4f}}<extra></extra>'
        ), row=row_number, col=1)

# ============ MAIN CALLBACK ============
@app.callback(
    Output('btc-indicator-graph', 'figure'),
    [Input('btc-scale', 'value'),
     Input('y-scale', 'value'),
     Input('indicator-row-2', 'value'),
     Input('indicator-row-3', 'value'),
     Input('indicator-row-4', 'value')]
)
def update_chart_figure(bitcoin_price_scale, y_axis_scale, row2_indicator, row3_indicator, row4_indicator):
    """Update chart based on user selections"""
    
    # Create subplot structure
    chart_figure = make_subplots(
        rows=4, cols=1, 
        shared_xaxes=True, 
        vertical_spacing=0.05,
        row_heights=[0.4, 0.2, 0.2, 0.2],
        subplot_titles=(
            'Bitcoin Price (with Halving Events)',
            'Indicator Row 2', 
            'Indicator Row 3', 
            'Indicator Row 4'
        )
    )
    
    # Add all traces
    add_bitcoin_price_trace(chart_figure, bitcoin_price_scale)
    add_halving_event_lines(chart_figure)
    
    for row_num, indicator_val in zip([2, 3, 4], [row2_indicator, row3_indicator, row4_indicator]):
        add_indicator_trace_or_placeholder(chart_figure, row_num, indicator_val)
    
    # Configure y-axis for Bitcoin price (top row)
    chart_figure.update_yaxes(
        title_text="<b>BTC Price (USD)</b>", 
        type=y_axis_scale, 
        row=1, 
        col=1
    )

    # Configure y-axes for indicator rows (rows 2-4)
    for row_num, indicator_val in zip([2, 3, 4], [row2_indicator, row3_indicator, row4_indicator]):
        if indicator_val != 'None':
            indicator_name, indicator_type = indicator_val.rsplit('_', 1)
            
            if indicator_type == 'abs':
                indicator_data = absolute_values[indicator_name]
                min_value = indicator_data.min() * 0.9 if indicator_data.min() > 0 else indicator_data.min() * 1.1
                max_value = indicator_data.max() * 1.1
            else:
                min_value, max_value = -0.05, 1.05
            
            chart_figure.update_yaxes(
                title_text=f"<b>{indicator_name}</b>",
                type='linear',
                range=[min_value, max_value],
                row=row_num,
                col=1
            )
        else:
            chart_figure.update_yaxes(
                title_text="<b>Indicator Value</b>",
                type='linear',
                row=row_num,
                col=1
            )
    
    # Configure x-axes
    chart_figure.update_xaxes(
        showline=True, 
        showticklabels=True, 
        ticks='outside', 
        tickangle=0
    )
    chart_figure.update_xaxes(title_text="<b>Date</b>", row=4, col=1)
    
    # Add rangeslider and range selector to bottom subplot
    chart_figure.update_xaxes(
        rangeslider=dict(
            visible=True, 
            thickness=0.05,
            bgcolor='rgba(200, 200, 200, 1)',
            bordercolor='#888', 
            borderwidth=2
        ),
        rangeselector=dict(
            buttons=[
                dict(count=1, label='1M', step='month', stepmode='backward'),
                dict(count=6, label='6M', step='month', stepmode='backward'),
                dict(count=1, label='1Y', step='year', stepmode='backward'),
                dict(count=2, label='2Y', step='year', stepmode='backward'),
                dict(step='all', label='All')
            ],
            x=0.0, 
            xanchor='left', 
            yanchor='top'
        ),
        row=4, 
        col=1
    )
    
    # Final layout configuration
    chart_figure.update_layout(
        hovermode='x unified',
        height=1200,
        template='plotly_white',
        showlegend=True,
        xaxis={'matches': 'x4'},   # Link all x-axes to bottom subplot
        xaxis2={'matches': 'x4'},
        xaxis3={'matches': 'x4'}
    )
    
    return chart_figure

# ============ RUN APP ============
if __name__ == '__main__':
    app.run(jupyter_mode='tab')
    
# ============ SAVE STATIC HTML ============
# Generate the figure with default settings
static_figure = update_chart_figure(
    bitcoin_price_scale='abs',
    y_axis_scale='log',
    row2_indicator='RSI_abs',
    row3_indicator='MACD_abs',
    row4_indicator='Fear_and_Greed_Index_abs'
)

# Save to HTML file (overwrites each time)
output_path = 'dashboard.html'
static_figure.write_html(
    output_path,
    config={'displayModeBar': True, 'scrollZoom': True}
)